In [1]:
import pandas as pd
import os
import json
import ast
from utils.statistics_random import *

In [2]:
pairs = pd.read_csv("trait_condition_pairs.csv")

In [3]:
all_traits = pd.read_csv("all_traits.csv")["Trait"].tolist()
all_traits = [normalize_trait(at) for at in all_traits]

In [4]:
rel = pd.read_csv("trait_related_genes.csv")
rel['Related_Genes'] = rel['Related_Genes'].apply(ast.literal_eval)
t2g = pd.Series(rel['Related_Genes'].values, index=rel['Trait']).to_dict()  # the mapping from trait to genes

In [ ]:
gene_info_path = './trait_related_genes.csv'
data_root = '/home/techt/Desktop/a4s/gold_subset'
output_root = './output_lasso_e-5'

condition = None

choose_lasso = 0
choose_lmm = 0
for trait in all_traits:
    print(f"Trait {trait} only")
    output_dir = os.path.join(output_root, trait)
    os.makedirs(output_dir, exist_ok=True)
    try:
        trait_data, _, _ = select_and_load_cohort(data_root, trait, is_two_step=False)
        trait_data = trait_data.drop(columns=['Age', 'Gender'], errors="ignore")

        Y = trait_data[trait].values
        X = trait_data.drop(columns=[trait]).values

        has_batch_effect = detect_batch_effect(X)
        if False and has_batch_effect:
            model_constructor = LMM
            choose_lmm += 1
        else:
            model_constructor = Lasso
            choose_lasso += 1

        param_values = [1e-5] # [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10]
        best_config, best_performance = tune_hyperparameters(model_constructor, param_values, X, Y, trait_data.columns, trait, gene_info_path, condition)
        model = ResidualizationRegressor(model_constructor, best_config)
        normalized_X, _ = normalize_data(X)
        model.fit(normalized_X, Y)

        var_names = trait_data.columns.tolist()
        significant_genes = interpret_result(model, var_names, trait, condition)
        save_result(significant_genes, best_performance, output_dir)

    except:
        print(f"Error processing trait {trait}")
        continue

Trait Breast_Cancer only
The cross-validation performance: {'prediction': {'accuracy': 99.17999999999999, 'precision': 99.726, 'recall': 99.36800000000001, 'f1': 99.546}, 'selection': {'precision': 22.404000000000003, 'precision_at_50': 30.4, 'recall': 12.352, 'f1': 15.922, 'jaccard': 8.652, 'jaccard2': 6.862}}
Trait Epilepsy only
The cross-validation performance: {'prediction': {'accuracy': 80.0, 'precision': 70.0, 'recall': 80.0, 'f1': 73.334}, 'selection': {'precision': 5.3919999999999995, 'precision_at_50': 5.6, 'recall': 1.052, 'f1': 1.7580000000000002, 'jaccard': 0.8859999999999999, 'jaccard2': 1.016}}
Trait Atherosclerosis only
Error processing trait Atherosclerosis
Trait Blood_Type only
Error processing trait Blood_Type
Trait Charcot-Marie-Tooth_Disease only
Error processing trait Charcot-Marie-Tooth_Disease
Trait Colon_and_Rectal_Cancer only
The cross-validation performance: {'prediction': {'accuracy': 99.304, 'precision': 99.718, 'recall': 99.484, 'f1': 99.598}, 'selection': 

In [ ]:
choose_lmm, choose_lasso

In [ ]:
gene_info_path = './trait_related_genes.csv'
data_root = '/home/techt/Desktop/a4s/gold_subset'
output_root = './output_lasso3_e-5'

choose_lasso2 = 0
choose_lmm2 = 0
for i, (index, row) in enumerate(pairs.iterrows()):
    try:
        trait, condition = row['Trait'], row['Condition']
        output_dir = os.path.join(output_root, trait)
        os.makedirs(output_dir, exist_ok=True)

        if condition in ['Age', 'Gender']:
            trait_data, _, _ = select_and_load_cohort(data_root, trait, condition, is_two_step=False)
            redundant_col = 'Age' if condition == 'Gender' else 'Gender'
            if redundant_col in trait_data.columns:
                trait_data = trait_data.drop(columns=[redundant_col])
        else:
            trait_data, condition_data, regressors = select_and_load_cohort(data_root, trait, condition, is_two_step=True, gene_info_path=gene_info_path)
            if regressors is None:
                print(f'No gene regressors for trait {trait} and condition {condition}')
                continue

            print("Common gene regressors for condition and trait", regressors)
            X_condition = condition_data[regressors].values
            Y_condition = condition_data[condition].values

            condition_type = 'binary' if len(np.unique(Y_condition)) == 2 else 'continuous'

            if condition_type == 'binary':
                if X_condition.shape[1] > X_condition.shape[0]:
                    model = LogisticRegression(penalty='l1', solver='liblinear', random_state=42)
                else:
                    model = LogisticRegression()
            else:
                if X_condition.shape[1] > X_condition.shape[0]:
                    model = Lasso()
                else:
                    model = LinearRegression()

            normalized_X_condition, _ = normalize_data(X_condition)
            model.fit(normalized_X_condition, Y_condition)

            regressors_in_trait = trait_data[regressors].values
            normalized_regressors_in_trait, _ = normalize_data(regressors_in_trait)
            if condition_type == 'binary':
                predicted_condition = model.predict_proba(normalized_regressors_in_trait)[:, 1]
            else:
                predicted_condition = model.predict(normalized_regressors_in_trait)

            trait_data[condition] = predicted_condition
            trait_data = trait_data.drop(columns=regressors)
            trait_data = trait_data.drop(columns=['Age', 'Gender'], errors='ignore')

        Y = trait_data[trait].values
        Z = trait_data[condition].values
        X = trait_data.drop(columns=[trait, condition]).values

        has_batch_effect = detect_batch_effect(X)
        if False and has_batch_effect:
            model_constructor = LMM
            choose_lmm2 += 1
        else:
            model_constructor = Lasso
            choose_lasso2 += 1

        param_values = [1e-5] # [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10]
        best_config, best_performance = tune_hyperparameters(model_constructor, param_values, X, Y, trait_data.columns, trait, gene_info_path, condition, Z)

        model = ResidualizationRegressor(model_constructor, best_config)
        normalized_X, _ = normalize_data(X)
        normalized_Z, _ = normalize_data(Z)
        model.fit(normalized_X, Y, normalized_Z)

        var_names = trait_data.columns.tolist()
        significant_genes = interpret_result(model, var_names, trait, condition)
        save_result(significant_genes, best_performance, output_dir, condition)
    except Exception as e:
        print(f"Error processing row {i}, for the trait '{trait}' and the condition '{condition}'\n: {e}")
        continue


In [ ]:
choose_lasso, choose_lasso2

In [ ]:
choose_lmm, choose_lmm2

In [ ]:
29 + 84 + 16 + 19

In [ ]:
import numpy as np

In [ ]:
param_values = np.logspace(-5, 1, base=10, num=13).tolist()

In [ ]:
param_values